# Download street network

Download street network data from Overture Maps.

In [4]:
from core.generate_streets import read_overture_region_streets

import glob

import geopandas as gpd
import datetime

In [5]:
streets_dir = "/data/uscuni-ulce/overture_streets/"
regions_datadir = "/data/uscuni-ulce/"

In [6]:
regions_v = "3"

In [7]:
country = 'fr_sp_nl_be'

## Run in parallel for all non-procesed streets in region_hulls

In [13]:
region_hulls = gpd.read_parquet('/data/uscuni-ulce/' + "regions/" + f"{country}_regions_hull.parquet")

In [31]:
processed_region_ids = [
    int(s.split(".")[0].split("_")[-1]) for s in glob.glob(streets_dir + "*")
]
region_hulls = region_hulls[~region_hulls.index.isin(processed_region_ids)]

In [32]:
region_hulls

,convex_hull
labels,
352338,"POLYGON ((3622100 2435000, 3615900 2436500, 36..."
354479,"POLYGON ((3656700 2388100, 3651000 2388200, 36..."
354704,"POLYGON ((3629200 2971300, 3623400 2975000, 36..."
354786,"POLYGON ((3648900 2327300, 3643900 2329100, 36..."
355019,"POLYGON ((3616100 2169000, 3601600 2170300, 35..."
...,...
534115,"POLYGON ((4164800 2843200, 4146900 2851600, 41..."
534412,"POLYGON ((4203400 2086900, 4201600 2117000, 42..."
534524,"POLYGON ((4258700 2026300, 4245100 2032800, 42..."


In [33]:
def process_and_save(region_hull, region_id):
    print("Processing", region_id, datetime.datetime.now())
    streets = read_overture_region_streets(region_hull, region_id)
    streets.to_parquet(streets_dir + f"streets_{region_id}.pq")

In [34]:
%%capture cap

from joblib import Parallel, delayed

n_jobs = -1
new = Parallel(n_jobs=n_jobs)(
    delayed(process_and_save)(region_hull.iloc[0], region_id)
    for region_id, region_hull in region_hulls.to_crs(epsg=4326).iterrows()
)

Processing 357190 2025-10-05 18:47:08.229496
Processing 377151 2025-10-05 18:58:08.104053
Processing 403958 2025-10-05 19:12:55.039626
Processing 362216 2025-10-05 18:47:08.243601
Processing 389001 2025-10-05 19:02:14.057676
Processing 407659 2025-10-05 19:16:37.505974
Processing 425100 2025-10-05 19:31:23.939978
Processing 436461 2025-10-05 19:42:52.775108
Processing 362823 2025-10-05 18:47:08.256934
Processing 379354 2025-10-05 18:58:24.064998
Processing 389826 2025-10-05 19:09:18.288506
Processing 408595 2025-10-05 19:21:35.840788
Processing 425225 2025-10-05 19:31:52.891830
Processing 437629 2025-10-05 19:46:42.934558
Processing 450071 2025-10-05 19:57:41.944568
Processing 354704 2025-10-05 18:47:08.229696
Processing 378294 2025-10-05 18:58:16.097722
Processing 397886 2025-10-05 19:11:39.574341
Processing 418544 2025-10-05 19:25:46.051384
Processing 430772 2025-10-05 19:37:04.689993
Processing 440010 2025-10-05 19:48:40.704709
Processing 454872 2025-10-05 19:59:49.023338
Processing

In [21]:
cap.stderr

'/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.\n  warnings.warn(\n'